# Tracing Without LangChain

LangSmith lets you instrument applications even if you don't want to depend on LangChain itself. This can help you debug, evaluate, and monotor any LLM application. In the following example, you will create an example series of LLM calls designed to generate, critique, and refine an argument. The whole series of calls will be traced to LangSmith, which will give you a run tree like the following:

<img src="./img/snapshot_1.png" alt="RunTree 1">

Before you get started, let's install some prerequisite packages. We'll need both the langsmith SDK and the OpenAI SDK.

In [1]:
%pip install -U pip > /dev/null
%pip install -U langsmith > /dev/null
%pip install -U openai > /dev/null

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, configure the API Key in the environment to make sure traces are logged to your account.

In [2]:
# %env LANGCHAIN_API_KEY=<your-api-key>
# %env LANGCHAIN_PROJECT=tracing-cookbook-tutorial

Next, define your chat application. Here we will use the `traceable` decorator.

In [3]:
import openai
import os
from datetime import datetime
from typing import List, Optional, Tuple
from langsmith.run_helpers import traceable


@traceable(run_type="llm")
def call_openai(data: List[dict], model: str = "gpt-3.5-turbo", temperature: float = 0.0):
    return openai.ChatCompletion.create(
        model=model,
        messages=data,
        temperature=temperature,
    )


@traceable(run_type="chain")
def argument_generator(query: str, additional_description: str = "") -> str:
    return call_openai(
        [
            {"role": "system", "content": f"You are a debater making an argument on a topic."
             f"{additional_description}"
             f" The current time is {datetime.now()}"},
            {"role": "user", "content": f"The discussion topic is {query}"}
        ]
    ).choices[0].message.content


@traceable(run_type="chain")
def critic(argument: str) -> str:
    return call_openai(
        [ 
            {"role": "system", "content": f"You are a critic."
           "\nWhat unresolved questions or criticism do you have after reading the following argument?"
            "Provide a concise summary of your feedback."},
            {"role": "system", "content": argument}
            
        ]
    ).choices[0].message.content


@traceable(run_type="chain")
def refiner(query: str, additional_description: str, current_arg: str, criticism: str) -> str:
    return call_openai(
        [
            {"role": "system", "content": f"You are a debater making an argument on a topic."
             f"{additional_description}"
             f" The current time is {datetime.now()}"},
            {"role": "user", "content": f"The discussion topic is {query}"},
            {"role": "assistant", "content": current_arg},
            {"role": "user", "content": criticism},
            {"role": "system", "content": "Please generate a new argument that incorporates the feedback from the user."}
        ]
    ).choices[0].message.content
    

@traceable(run_type="chain")      
def argument_chain(query: str, additional_description: str = "") -> str:
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    return refiner(query, additional_description, argument, criticism)

The @traceable decorator is experimental and will likely see breaking changes.


In [4]:
result = argument_chain(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)
print(result)

Sunshine, in moderation, is good for you because it provides essential vitamin D, which is crucial for bone health, immune function, and mental well-being. However, it is important to acknowledge the potential risks of excessive sun exposure, such as skin damage and an increased risk of skin cancer. By practicing sun safety measures, such as wearing sunscreen and seeking shade during peak hours, individuals can enjoy the benefits of sunshine while minimizing the potential drawbacks. Additionally, while there is anecdotal evidence suggesting that sunlight can improve mood and sleep patterns, further research is needed to fully understand and establish the causal relationship between sunlight and mental well-being.


## Capturing the runs

The `traceable` decorator will inject the current `RunTree` object into the traced function if the argument is provided. You can use this to do things like fetch the run ID. Below, our `Chat2` app is an identical to the previous one but views the injected run_tree so we can fetch the latest run. The chat 

In [5]:
from langsmith import RunTree
    
run_ids = []
    
@traceable(run_type="chain")      
def argument_chain2(query: str, run_tree: RunTree,additional_description: str = "") -> str:
    argument = argument_generator(query, additional_description)
    run_ids.append(run_tree.id)
    criticism = critic(argument)
    return refiner(query, additional_description, argument, criticism)       

In [6]:
result = argument_chain2(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)

In [7]:
latest_id = run_ids[-1]

In [17]:
from langsmith import Client

client = Client()
shared_link = client.share_run(latest_id)

<img src="img/snapshot_2.png" alt="RunTree 2">

In [18]:
shared_link

'https://dev.smith.langchain.com/public/6c28f481-044d-4be1-8ffe-c06e8773ea0a/r'

## Using the RunTree

The `traceable` decorator is lightweight but less flexible when it comes to defining the schema of the logs you want to save. Below, we will make an example of the chat application that logs runs using the RunTree object

In [10]:
def call_openai(data: List[dict], run_tree: RunTree,  model: str = "gpt-3.5-turbo", temperature: float = 0.0):
    child = run_tree.create_child(
        name="ChatOpenAI",
        inputs={"data": data, "model": model, "temperature": temperature},
        run_type="llm",
    )
    child.post()
    try:
        result = openai.ChatCompletion.create(
            model=model,
            messages=data,
            temperature=temperature,
        )
        child.end(outputs={"response": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result


def argument_generator(query: str, run_tree: RunTree, additional_description: str = "") -> str:
    child = run_tree.create_child(name="ArgumentGenerator", inputs={"query": query, additional_description: additional_description}, run_type="chain")
    child.post()
    try:
        result = call_openai(
            [
                {"role": "system", "content": f"You are a debater making an argument on a topic."
                    f"{additional_description}"
                    f" The current time is {datetime.now()}"},
                {"role": "user", "content": f"The discussion topic is {query}"}
            ], run_tree=child
        ).choices[0].message.content
        child.end(outputs={"argument": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result


def critic(argument: str, run_tree: RunTree) -> str:
    child = run_tree.create_child(name="Critic", run_type="chain", inputs={"argument": argument})
    child.post()
    try:
        result = call_openai(
            [
                {"role": "system", "content": f"You are a critic."
                    "\nWhat unresolved questions or criticism do you have after reading the following argument?"
                    "Provide a concise summary of your feedback."},
                {"role": "system", "content": argument}
            ], run_tree=child
        ).choices[0].message.content
        child.end(outputs={"criticism": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result


def refiner(query: str, additional_description: str, current_arg: str, criticism: str, run_tree: RunTree) -> str:
    child = run_tree.create_child(name="Refiner", run_type="chain", inputs={"query": query, "additional_description": additional_description})
    child.post()
    try:
        result = call_openai(
            [
                {"role": "system", "content": f"You are a debater making an argument on a topic."
                    f"{additional_description}"
                    f" The current time is {datetime.now()}"},
                {"role": "user", "content": f"The discussion topic is {query}"},
                {"role": "assistant", "content": current_arg},
                {"role": "user", "content": criticism},
                {"role": "system", "content": "Please generate a new argument that incorporates the feedback from the user."}
            ], run_tree=child
        ).choices[0].message.content
        child.end(outputs={"refined_argument": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result
    

run_tree_run_ids = []
def argument_chain3(query: str, additional_description: str = "", run_tree: RunTree = None) -> str:
    inputs = {
        "query": query,
        "additional_description": additional_description
    }
    if run_tree is None:
        argument_run_tree = RunTree(name="ArgumentChain", run_type="chain", inputs=inputs)
    else:
        argument_run_tree = run_tree.create_child(name="ArgumentChain", run_type="chain", inputs=inputs)
    argument_run_tree.post()
    run_tree_run_ids.append(argument_run_tree.id)
    try:
        argument = argument_generator(query, additional_description=additional_description, run_tree=argument_run_tree)
        criticism = critic(argument, run_tree=argument_run_tree)
        result = refiner(query, additional_description, argument, criticism, run_tree=argument_run_tree)
        argument_run_tree.end(outputs={"final_argument": result})
    except Exception as e:
        argument_run_tree.end(error=str(e))
        raise
    finally:
        argument_run_tree.patch()
    return result

In [11]:
result = argument_chain3(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)

In [12]:
latest_id = run_tree_run_ids[-1]
shared_link = client.share_run(latest_id)
shared_link

'https://dev.smith.langchain.com/public/67d56aed-e25d-475a-9c56-4650c2f5394a/r'

<img src="img/snapshot_3.png" />